In [30]:
import sys
sys.path.insert(0, '../src')

import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from pathlib import Path
from colorama import Fore, Style
from torch.utils.data import Dataset, DataLoader
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from data.encode_data import encode_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score,
                             recall_score,
                             precision_score,
                             f1_score,
                             roc_auc_score)

sns.set_theme(style='white')

In [31]:
current_directory = Path.cwd()
RAW_DATA_PATH = current_directory.parent / 'data/raw/affair_data.csv'

In [32]:
raw_data = pd.read_csv(RAW_DATA_PATH)
raw_data.head()

,rate_marriage,age,yrs_married,children,religious,educ,occupation,occupation_husb,affairs
0,3.0,32.0,9.0,3.0,3.0,17.0,2.0,5.0,1
1,3.0,27.0,13.0,3.0,1.0,14.0,3.0,4.0,1
2,4.0,22.0,2.5,0.0,1.0,16.0,3.0,5.0,1
3,4.0,37.0,16.5,4.0,3.0,16.0,5.0,5.0,1
4,5.0,27.0,9.0,1.0,1.0,14.0,3.0,4.0,1


In [33]:
data_encoded = encode_dataset(raw_data)
data_encoded.head()

,rate_marriage,children,religious,educ,affairs,age_17.5,age_22.0,age_27.0,age_32.0,age_37.0,...,occupation_3.0,occupation_4.0,occupation_5.0,occupation_6.0,occupation_husb_1.0,occupation_husb_2.0,occupation_husb_3.0,occupation_husb_4.0,occupation_husb_5.0,occupation_husb_6.0
0,3.0,3.0,3.0,17.0,1,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,3.0,3.0,1.0,14.0,1,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,4.0,0.0,1.0,16.0,1,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,4.0,4.0,3.0,16.0,1,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,5.0,1.0,1.0,14.0,1,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [44]:
X = data_encoded.drop(columns='affairs')
y = data_encoded['affairs']


class NeuralNet(nn.Module):
    def __init__(self, input_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x
    

def train_and_evaluate_models(X, y):
    # Разделение данных на обучающий и тестовый набoры
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Инициализация моделей
    models = {
        "Logistic Regression": LogisticRegression(max_iter=2000),
        "Random Forest": RandomForestClassifier(),
        "Decision Tree": DecisionTreeClassifier(),
        "CatBoost": CatBoostClassifier(iterations=2000, verbose=0),
        "XGBoost": XGBClassifier(),
        "Neural Network": NeuralNet(input_size=X_train.shape[1]).to(device)

    }
    class CustomDataset(Dataset):
        def __init__(self, X_data, y_data):
            self.X_data = X_data
            self.y_data = y_data

        def __getitem__(self, index):
            return self.X_data[index], self.y_data[index]

        def __len__(self):
            return len(self.X_data)

    train_dataset = CustomDataset(torch.FloatTensor(X_train.values), torch.FloatTensor(y_train.values))
    train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

    # Инициализация списка для хранения результатов
    results = []
    
    # Обучение и оценка каждой модели
    for name, model in models.items():
        if name == "Neural Network":
            criterion = nn.BCELoss()
            optimizer = optim.Adam(model.parameters(), lr=0.001)

            model.train()
            for epoch in range(50):
                for X_batch, y_batch in train_loader:
                    # Перемещение данных на устройство
                    X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                    
                    optimizer.zero_grad()
                    output = model(X_batch)
                    loss = criterion(output, y_batch.unsqueeze(1))
                    loss.backward()
                    optimizer.step()

            model.eval()
            with torch.no_grad():
                # Перемещение данных на устройство и предсказание
                y_pred = model(torch.FloatTensor(X_test.values).to(device)).squeeze().round().cpu().numpy()
        else:
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
        
        accuracy = accuracy_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        auc = roc_auc_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        
        results.append({'Model': name, 'Accuracy': accuracy, 'Recall': recall,
                        'Precision': precision, 'AUC': auc, 'F1-score': f1})
    
    # Преобразование списка результатов в DataFrame
    results_df = pd.DataFrame(results)
    plt.figure(figsize=(10, 6))
    plt.axis('off')
    plt.table(cellText=results_df.values,
              colLabels=results_df.columns,
              cellLoc = 'center', loc='upper left')
    plt.tight_layout()
    plt.savefig('../reports/figures/model_results.png', dpi=300, bbox_inches='tight', pad_inches=0.5)
    plt.close()
    return results_df.sort_values(by='AUC', ascending=False)

In [45]:
train_and_evaluate_models(X, y)

,Model,Accuracy,Recall,Precision,AUC,F1-score
5,Neural Network,0.704082,0.401460,0.557432,0.624832,0.466761
0,Logistic Regression,0.707221,0.391727,0.566901,0.624601,0.463309
3,CatBoost,0.688383,0.403893,0.522013,0.613882,0.455418
4,XGBoost,0.672684,0.416058,0.491379,0.605480,0.450593
1,Random Forest,0.661695,0.399027,0.471264,0.592909,0.432148
2,Decision Tree,0.635793,0.413625,0.432570,0.577612,0.422886


Лучше всего себя показала NeuralNet. Тюнить и использовать будем её.